In [1]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

import matplotlib.animation as animation
from matplotlib import style
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime
import tkinter as tk
from tkinter import ttk
from pandastable import Table, TableModel

In [2]:
df = pd.read_csv("prices.csv")
securities = pd.read_csv("securities.csv")
sector = dict(zip(securities['Ticker symbol'], securities['GICS Sector']))
company = dict(zip(securities['Ticker symbol'], securities['Security']))
df['sector'] = df['symbol'].map(sector)
df['name'] = df['symbol'].map(company)
IT_df = df.loc[df['sector'] == "Information Technology"]
stock_open = np.array(IT_df['open']).T 
stock_close = np.array(IT_df['close']).T
movements = stock_close - stock_open
IT_df["movement"] = movements


IT_df_clean = IT_df.pivot_table(index='name',columns ='date',values ='movement', fill_value=0)
filtered_df = IT_df.drop(["low",'high','volume','sector','name','movement'], axis=1)


In [3]:
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
model = LinearRegression()
MSFTDF = filtered_df.loc[filtered_df['symbol']=="MSFT"]
LR_MSFT = MSFTDF.drop(['symbol'], axis = 1)
LR_MSFT.reset_index(drop=True, inplace=True)
LR_MSFT.index += 1
train, test = train_test_split(LR_MSFT, test_size = 0.20)
X_train = np.array(train.index).reshape(-1,1)
y_train = train['close']
model.fit(X_train, y_train)
LR_MSFT['Prediction'] = model.predict(np.array(LR_MSFT.index).reshape(-1, 1))








from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 

train, test = train_test_split(LR_MSFT, test_size = 0.20)
X_train = np.array(train.index).reshape(-1,1)
y_train = train['close']

In [4]:
self = tk.Tk()   

self.geometry("1000x1000")
self.title("Application")
        
#frames creation
frame = ttk.Frame(self)
frame2 = ttk.Frame(self) 
frame3 = ttk.Frame(self)
frame.grid()
frame2.grid()
frame3.grid()

#label for stock symbol text box
stocksymbol = tk.StringVar()
stocksymbol_label = ttk.Label(frame, text="Input Stock Symbol:",style="Others.TLabel")
stocksymbol_label.grid(column=0, row=0, padx=10) #, pady=10)
            
def search_df(*event):
    search_result = filtered_df.loc[filtered_df['symbol'].str.contains(stocksymbol.get().strip(),na=False,case=False)]
    self.table = pt = Table(frame3, dataframe=search_result, showtoolbar=False, showstatusbar=False, width=500, height=500)
    pt.show()
    
stocksymbol_entry = ttk.Entry(frame, textvariable=stocksymbol)
stocksymbol_entry.grid(column=1, row=0) 

def plot():
    win2 = tk.Tk()
    fig = Figure(figsize = (5,5), dpi = 100)
    search_result = filtered_df.loc[filtered_df['symbol'].str.contains(stocksymbol.get().strip(),na=False,case=False)]
    search_value = stocksymbol.get().upper()
    a = fig.add_subplot(111)
    search_result.reset_index(drop=True, inplace=True)
    search_result.index += 1
    a.plot(search_result['open'],label = 'open') # Opening prices of first 30 days are plotted against date
    a.plot(search_result['close'],label = 'close')
    a.set_xlabel('Date',fontsize = 10)
    a.set_ylabel('Price',fontsize = 10)
    a.set_title("Open and Close Over Time for " + search_value)
    a.legend()
    a.grid()
    canv = FigureCanvasTkAgg(fig, master = win2)
    canv.draw()
    get_widz = canv.get_tk_widget()
    get_widz.pack()
    win2.mainloop()
    
def predict():
    win3 = tk.Tk()
    fig = Figure(figsize = (5,5), dpi = 100)
    a = fig.add_subplot(111)
    search_result = filtered_df.loc[filtered_df['symbol'].str.contains(stocksymbol.get().strip(),na=False,case=False)]
    search_value = stocksymbol.get().upper()
    a.set_title("Linear Regression: Time vs. Price for " + search_value)
    search_result.reset_index(drop=True, inplace=True)
    #search_result.strip()
    search_result.index += 1
    train, test = train_test_split(search_result, test_size = 0.3)
    X_train = np.array(train.index).reshape(-1,1)
    y_train = train['close']
    a.scatter(X_train, y_train, edgecolor='w', label='Actual Price')
    a.plot(X_train, model.predict(X_train), color='k', label='Predicted Price')
    a.set_xlabel('Days',fontsize=10)
    a.set_ylabel('Stock Price',fontsize=10)
    a.legend()
    canv = FigureCanvasTkAgg(fig, master = win3)
    canv.draw()
    get_widz = canv.get_tk_widget()
    get_widz.pack()
    win3.mainloop()

#search_button
search_button = ttk.Button(frame2, text='Search',command=search_df)
search_button.grid(column=0, row=2, padx=10)

#plot button
plot_button = ttk.Button(frame2, text='Plot',command=plot)
plot_button.grid(column=1, row=2, padx=10, pady=10)

#plot2 button
plot_button2 = ttk.Button(frame2, text='Predict',command=predict)
plot_button2.grid(column=2, row=2, padx=10, pady=10)

#quit button
quit_button = ttk.Button(frame2, text='Quit', command=self.destroy)
quit_button.grid(column=3, row=2, padx=10, pady=10)  

#style of elements
style = ttk.Style(self)
style.configure('Top.TLabel', font=("Times New Roman", 16))
style.configure('Others.TLabel', font=("Times New Roman",12))
style.configure('TButton', font=('Times New Roman', 12))
style.map('TButton',background=[('pressed', 'white'),('active', 'grey')])
    
#run
self.mainloop()